In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

# Scraping with BS4

In [2]:
url = 'http://www.xeno-canto.org/species/Cercomacra-tyrannina?pg=1&dir=0&order=typ'
soup = BeautifulSoup(requests.get(url).text, 'lxml')

In [3]:
soup_table = soup.find(class_="results")

In [4]:
icons = soup_table.find_all('td')

In [5]:
print icons[11]

<td><a download="XC18190 - Dusky Antbird - Cercomacra tyrannina.mp3" href="/18190/download"><img class="icon" height="14" src="/static/img/download.png" title="Download file 'Cercomacra tyrannina_1055_05208.mp3'" width="14"/></a><a class="tooltip" href="/new_thread.php?frontID=18190" title="Open a discussion, mail the recordist"><img alt="discuss icon" class="icon" height="14" src="/static/img/discuss.png" width="14"/></a><div class="rating">
<ul>
<li class="" id="rating-18190-1"><span>A</span></li><li class="" id="rating-18190-2"><span>B</span></li><li class="selected" id="rating-18190-3"><span>C</span></li><li class="" id="rating-18190-4"><span>D</span></li><li class="" id="rating-18190-5"><span>E</span></li>
</ul>
</div></td>


In [6]:
print icons[11].find('a')['href']

/18190/download


In [7]:
# got it working - now to make it fully automated for song only

# Bringing it all home

In [8]:
# using the following birds based on previous research and geographical interest
names = ['dusky_antbird', 'great_antshrike', 'barred_antshrike', 'purple_finch', 'northern_cardinal', 
         'black-capped_chickadee', 'american_goldfinch', 'tufted_titmouse', 'american_robin', 'baltimore_oriole', 
         'bluejay', 'bluebird', 'house_finch']

urls = ['http://www.xeno-canto.org/species/Cercomacra-tyrannina?pg=1', 
        'http://www.xeno-canto.org/species/Taraba-major?pg=1', 
        'http://www.xeno-canto.org/species/Thamnophilus-doliatus?pg=1', 
        'http://www.xeno-canto.org/species/Haemorhous-purpureus?pg=1', 
        'http://www.xeno-canto.org/species/Cardinalis-cardinalis?pg=1', 
        'http://www.xeno-canto.org/species/Poecile-atricapillus?pg=1', 
        'http://www.xeno-canto.org/species/Spinus-tristis?pg=1', 
        'http://www.xeno-canto.org/species/Baeolophus-bicolor?pg=1', 
        'http://www.xeno-canto.org/species/Turdus-migratorius?pg=1', 
        'http://www.xeno-canto.org/species/Icterus-galbula?pg=1', 
        'http://www.xeno-canto.org/species/Cyanocitta-cristata?pg=1', 
        'http://www.xeno-canto.org/species/Sialia-sialis?pg=1', 
        'http://www.xeno-canto.org/species/Haemorhous-mexicanus?pg=1'
       ]

all_urls = []

In [9]:
for url in urls:
    soup = BeautifulSoup(requests.get(url).text, 'lxml')
    pages = soup.find(class_='results-pages')
    last_page = int(pages.find_all('li')[-2].text)
    all_urls.append([url[:-1]+str(i) for i in range(1,last_page+1)])

In [10]:
# check we have correct urls

all_urls[1]

['http://www.xeno-canto.org/species/Taraba-major?pg=1',
 'http://www.xeno-canto.org/species/Taraba-major?pg=2',
 'http://www.xeno-canto.org/species/Taraba-major?pg=3',
 'http://www.xeno-canto.org/species/Taraba-major?pg=4',
 'http://www.xeno-canto.org/species/Taraba-major?pg=5',
 'http://www.xeno-canto.org/species/Taraba-major?pg=6',
 'http://www.xeno-canto.org/species/Taraba-major?pg=7',
 'http://www.xeno-canto.org/species/Taraba-major?pg=8',
 'http://www.xeno-canto.org/species/Taraba-major?pg=9']

In [ ]:
# get download links

download_links = []
for species in all_urls:
    download_links.append([])
    for page in species:
        soup = BeautifulSoup(requests.get(page).text, 'lxml')
        soup_table = soup.find(class_="results")
        soup_rows = soup_table.find_all('tr')
        for soup_row in soup_rows[1:]:
            elements = soup_row.find_all('td')
            if elements[9].text.lower() == 'song ':
                download_links[-1].append('http://www.xeno-canto.org'+elements[11].find('a')['href'])   

In [ ]:
# write links to to file

for species in download_links:
    with open('bird_links/'+names[download_links.index(species)]+'.txt', 'w') as f:
        for link in species:
            f.write(link+'\n')

In [11]:
# now modify so that we find only those with 'A' rating

download_links = []
for species in all_urls:
    download_links.append([])
    for page in species:
        soup = BeautifulSoup(requests.get(page).text, 'lxml')
        soup_table = soup.find(class_="results")
        soup_rows = soup_table.find_all('tr')
        for soup_row in soup_rows[1:]:
            elements = soup_row.find_all('td')
            if elements[11].find(class_='selected') == None:
                continue
            if elements[9].text.lower() == 'song ' and elements[11].find(class_='selected').text == 'A':
                download_links[-1].append('http://www.xeno-canto.org'+elements[11].find('a')['href'])

In [12]:
# check how many recordings we have for each bird

for i in range(len(download_links)):
    print names[i], len(download_links[i])

dusky_antbird 39
great_antshrike 57
barred_antshrike 41
purple_finch 12
northern_cardinal 47
black-capped_chickadee 16
american_goldfinch 14
tufted_titmouse 25
american_robin 39
baltimore_oriole 9
bluejay 2
bluebird 9
house_finch 27


In [19]:
# condense so we only have the top 6 birds
# write links to file so we can wget to download

zipped = zip(names, download_links)
cond_download_links = sorted(zipped, key=lambda x: len(x[1]), reverse=True)[:6]

for species in cond_download_links:
    with open('bird_links/A-bird_links/'+species[0]+'.txt', 'w') as f:
        for link in species[1]:
            f.write(link+'\n')

In [22]:
# condense to only take 6 birds with top number of 'A' recordings 
# use wget to download the mp3 files

for species in cond_download_links:
    # this will use the wget bash command to download the links in the textfile and save it to the location specified
    # !wget --content-disposition 'bird_links/A-bird_links/'+species[0]+'.txt' -P 'bird_audio/A_rate_only/mp3/A-bird'+str(birdnum)
    print 'Finished downloading ' + species[0]

Finished downloading great_antshrike
Finished downloading northern_cardinal
Finished downloading barred_antshrike
Finished downloading dusky_antbird
Finished downloading american_robin
Finished downloading house_finch


In [25]:
# take input from the 'A' mp3 file directories and convert to wav files

from pydub import AudioSegment
import os

for i in os.listdir(os.path.abspath('bird_audio/A_rate_only/mp3/')):
    if i[0] == 'A':
        for song in os.listdir(os.path.abspath('bird_audio/A_rate_only/mp3/'+i)):
            if song[0] == '.':
                continue
            sound = AudioSegment.from_mp3(os.path.abspath('bird_audio/A_rate_only/mp3/'+i+'/'+song))
            sound.export(os.path.abspath('bird_audio/A_rate_only/wav/wav_'+i+'/'+song[:-4]+'.wav'), format="wav")